### Geometry

First, we'll load in a GeoJSON file describing the outline of Pine Island Ice Shelf.
This outline was hand-digitized from the various input data sets we'll use in a GIS.

In [ ]:
import geojson
outline_filename = 'pine-island-shelf.geojson'
with open(outline_filename, 'r') as outline_file:
    outline = geojson.load(outline_file)

Next we'll use a few utility functions from icepack to turn this outline into the input format for the mesh generator [gmsh](https://www.gmsh.info).
We can then generate an unstructured triangular mesh of the domain and load in that mesh.

In [ ]:
import icepack
geometry = icepack.meshing.collection_to_geo(outline)
with open('pine-island-shelf.geo', 'w') as geo_file:
    geo_file.write(geometry.get_code())

In [ ]:
!gmsh -2 -format msh2 -v 2 -o pine-island-shelf.msh pine-island-shelf.geo

In [ ]:
import firedrake
mesh = firedrake.Mesh('pine-island-shelf.msh')

The colors correspond to the numeric IDs of each boundary segment (note the legend in the corner).
We need a way of identifying different boundary conditions in the ice shelf flow model in order to determine where ice is flowing in from and where the terminus is.

In [ ]:
import icepack.plot
fig, axes = icepack.plot.subplots()
icepack.plot.triplot(mesh, axes=axes, boundary_kw={'colors': ['r', 'g', 'b', 'k']})
axes.legend();

### Ice shelf draft

Next we have to load in some observational data for the ice shelf draft.
The demos for icepack include a module to fetch the most common observational data sets.
We'll use BedMachine Antarctica because they've already done the hard work of things like firn and geoid corrections to ice shelf thickness.
This routine will download the BedMachine dataset from NSIDC.
If this is your first time running this notebook, you'll be prompted for your EarthData username and password.

In [ ]:
bedmachine_filename = icepack.datasets.fetch_bedmachine_antarctica()

To get the ice shelf draft, we'll first get the surface elevation and thickness of the ice shelf.

In [ ]:
import rasterio
surface_grid = rasterio.open('netcdf:' + bedmachine_filename + ':surface', 'r')
thickness_grid = rasterio.open('netcdf:' + bedmachine_filename + ':thickness', 'r')

We'll represent the ice shelf draft using continuous, piecewise quadratic basis functions.

In [ ]:
Z = firedrake.FunctionSpace(mesh, family='CG', degree=2)

Next we'll interpolate the gridded data to our finite element space.

In [ ]:
surface = icepack.interpolate(surface_grid, Z)
thickness = icepack.interpolate(thickness_grid, Z)

Finally we can get the ice shelf draft as the difference of the surface and the thickness.

In [ ]:
z_obs = firedrake.interpolate(surface - thickness, Z)

In [ ]:
fig, axes = icepack.plot.subplots()
contours = icepack.plot.tricontourf(z_obs, 40, axes=axes)
fig.colorbar(contours);

In the interest of keeping our numerical solver from losing its bloody mind, we'll smooth over the ice shelf.
The spin-up of the plume will proceed in two stages.
First, we'll use a very smooth ice shelf draft.
Most of the interesting features, like sub-ice shelf channels, will have been diffused out.
But this will make it easier to evolve the plume towards a sane steady state than if we had used the real data as-is.
In the next phase, we'll slowly morph the ice shelf draft towards a value with a much lower smoothing length in order to recapture more of the features of the real data.

In [ ]:
from firedrake import inner, grad, dx, ds, Constant
z = firedrake.Function(Z)
α = Constant(8e3)

parameters = {
    'solver_parameters': {
        'ksp_type': 'preonly',
        'pc_type': 'lu',
        'pc_factor_mat_solver_type': 'mumps'
    }
}

J = 0.5 * ((z - z_obs)**2 + α**2 * inner(grad(z), grad(z))) * dx
F = firedrake.derivative(J, z)
firedrake.solve(F == 0, z, **parameters)

This is enough smoothing to keep the plume solver from exploding, but not so much as to wipe out features like sub-ice shelf channels.

In [ ]:
fig, axes = icepack.plot.subplots()
contours = icepack.plot.tricontourf(z, 40, axes=axes)
fig.colorbar(contours);

Some amount of smoothing is also usually necessary for the ice flow model too.
Newer remote sensing platforms like ICESat-2 are sophisticated enough to resolve individual crevasses, introducing sharp breaks in the ice thickness.
The gradient of the ice thickness is one of the sources of the ice flow model, so these features, which are of too small a scale to really influence the flow by themselves, end up breaking the numerics.

### Background ocean

Next we have to decide on what the salinity and temperature of the background ocean will be.
More than any other embayment in Antartica, the Amundsen Sea is subject to intrusions of Circumpolar Deep Water, which is very warm and saline.
We'll take the background ocean to have a temperature of 1${}^\circ$C and salinity of 34.73 PSU.

In [ ]:
T_a = Constant(1.0)
s_a = Constant(34.73)

The plume water, by contrast, will have a temperature of around -3.5${}^\circ$C and a salinity of 0.
The temperature and salinity of the plume control how buoyant it is over the background ocean; the plume floats above the background ocean water because it has much less salt and is thus less dense.
The salinity of the plume will approach that of the background ocean as the plume entrains ocean water.
The addition of fresh meltwater from the ice shelf base will partly counteract this process.
In principle, the plume could destabilize at a point if it were to, say, obtain a salinity value close to or exceeding that of the background ocean.

### Inflow conditions

Christine Dow at the University of Waterloo has graciously provided us with some output from a hydrology model of the water flux out of the grounding line and into the plume.
The total water influx from the model across all channels is 35.64 m${}^3$/s.
The maximum influx of any one channel is 30.46 m${}^3$/s with a velocity of 0.86 m/s, so if you figure that the channel is shaped liked a half-circle, this shakes out to a radius of 4.75m.
The largest channel is centered at the point (x, y) = -1589${}^3$km, -255.5${}^3$km in this coordinate system, which is right near the deepest point of the grounding line.
There are two smaller channels about 6km away to either side.

We won't try to simulate these basal water channels as the point-like sources that they are because this is too hard.
Instead, we'll use an inflow plume velocity and thickness that reproduce the net flux from the hydrology model, but with a profile that's smoothed out over a few grid cells.

In [ ]:
max_inflow_point = Constant((-1589e3, -255.5e3))
radius = Constant(10e3)
grounding_line_flux = Constant(35.64)

In [ ]:
def sech(z):
    return 2 / (firedrake.exp(z) + firedrake.exp(-z))

n = firedrake.FacetNormal(mesh)
x = firedrake.SpatialCoordinate(mesh)
δx = x - max_inflow_point
r = firedrake.sqrt(inner(δx, δx))
shape = -sech(r / radius) * n
total = firedrake.assemble(inner(shape, -n) * ds((3,)))
q_in = Constant(grounding_line_flux / total) * shape

We'll make the inflow thickness have roughly the same profile.

In [ ]:
max_inflow_thickness = Constant(4.0)
D_in = max_inflow_thickness * sech(r / radius)

And finally the influx of thermal energy and salt will correspond to a temperature and salinity of -3.5${}^\circ$C and 0 PSU respectively.

In [ ]:
T_in = firedrake.Constant(-3.5)
s_in = firedrake.Constant(0.0)

E_in = D_in * T_in
S_in = D_in * s_in

### Initial state

Next, we need to come up with some vaguely sane initial state of the plume.
The model can quickly explode if we initialize it with a weird initial state that has large transients, so we need to find something vaguely reasonable.
Roughly speaking, the fluid velocity should align with the gradient of the ice shelf bottom.

In [ ]:
W = firedrake.VectorFunctionSpace(mesh, family='CG', degree=2)
grad_z = firedrake.project(grad(z), W)

To make the initial velocity field, we'll solve an optimization problem.
The objective functional will favor a velocity field that is:

1. smoothly varying
2. non-divergent
3. aligns with the ice shelf draft gradient whenever it points into the domain

Putting all of these criteria together, we get the objective 

$$J = \frac{\alpha^2}{2}\int_\Omega\left\{|\nabla w|^2 + \gamma^2(\nabla\cdot w)^2\right\}dx + \frac{\alpha}{2}\int_\Gamma |w - v|^2ds$$

where $\Gamma$ is everything except the ice shelf terminus, $v$ is the inflow vector

$$v = \max\{0, \nu\cdot\nabla z_b\}\nu,$$

and $\nu$ the unit inward-pointing normal to the domain.

In [ ]:
from firedrake import div
w = firedrake.Function(W)
γ = firedrake.Constant(10)

J = (
    0.5 * α**2 * (inner(grad(w), grad(w)) + γ**2 * div(w)**2) * dx +
    0.5 * α * inner(w - q_in, w - q_in) * ds((2, 3, 4)) -
    inner(w, grad_z) * dx
)
F = firedrake.derivative(J, w)
firedrake.solve(F == 0, w, **parameters)

As a sanity check we can make a stream plot of the initial value of the plume velocity.
The streamlines get thicker towards the end, so this vector field actually is going from the inflow boundary to the terminus as we had hoped.

In [ ]:
fig, axes = icepack.plot.subplots()
firedrake.triplot(
    mesh, axes=axes,
    boundary_kw={'color': 'k'},
    interior_kw={'linewidth': 0.05}
)
streamlines = firedrake.streamplot(w, resolution=2e3, seed=1, axes=axes)
fig.colorbar(streamlines);

Now we'll create function spaces where the solutions will live.
Once again we'll use DG(1) basis functions for the scalar fields and BDFM(2) for the momentum.
Then we'll project the vector field $w$ into the momentum space.

In [ ]:
scalar_element = firedrake.FiniteElement('DG', 'triangle', 1)
vector_element = firedrake.FiniteElement('BDFM', 'triangle', 2)
Q = firedrake.FunctionSpace(mesh, scalar_element)
V = firedrake.FunctionSpace(mesh, vector_element)

q = firedrake.project(w, V)

Once again we'll solve an optimization problem to come up with a vaguely sensible value of the initial plume thickness.
The objective functional favors smooth solutions that match the inflow thickness.

In [ ]:
from firedrake import dS
D = firedrake.Function(Q)
ℓ = firedrake.CellDiameter(mesh)
J = 0.5 * (
    α**2 * inner(grad(D), grad(D)) * dx +
    α**2 / (ℓ('+') + l('-')) * (D('+') - D('-'))**2 * dS +
    α * (D - D_in)**2 * ds((3,))
)

F = firedrake.derivative(J, D)
firedrake.solve(F == 0, D, **parameters)

In [ ]:
fig, axes = icepack.plot.subplots()
colors = icepack.plot.tripcolor(D, axes=axes)
fig.colorbar(colors);

We can be much more blunt about the initial values of the salinity and temperature of the plume -- the meltwater has effectively no salt content at all and is at the pressure melting point of that ice.
But remember that we're describing the system in terms of extensive quantities like momentum, thermal energy, and total salt mass, rather than intensive quantities like velocity, temperature, and salinity.

In [ ]:
E = firedrake.project(D * T_in, Q)
S = firedrake.project(D * s_in, Q)

Now we'll create a big mixed function space to represent the entire plume state and pack all of our initial data into a mixed function.
The ordering of the fields in the mixed function is thickness, momentum, energy, salt.

In [ ]:
Z = Q * V * Q * Q
fields = firedrake.Function(Z)
fields.sub(0).assign(D)
fields.sub(1).assign(q)
fields.sub(2).assign(E)
fields.sub(3).assign(S);

### Will it go?!

Now we're ready to actually start running things.
First, we'll pack all of the data about the background ocean and the boundary conditions into a dictionary, since we'll be passing these to many different functions.

In [ ]:
inputs = {
    'temperature_ambient': T_a,
    'salinity_ambient': s_a,
    'ice_shelf_draft': z,
    'thickness_in': D_in,
    'momentum_in': q_in,
    'energy_in': E_in,
    'salt_in': S_in,
    'inflow_ids': (3,),
    'outflow_ids': (1,),
}

Next, we need to create a function that calculates the residual of the model physics.

In [ ]:
from plumes.models import plume
class SimplePhysics(plume.PlumePhysics):
    def density_contrast(self, fields, **inputs):
        return Constant(0.0015)
    
physics = SimplePhysics()

def equation(fields):
    transport = plume.make_transport_equation(physics, **inputs)
    sources = plume.make_source_equation(physics, **inputs)
    sinks = plume.make_sink_equation(physics, **inputs)
    
    return transport(fields) + sources(fields) + sinks(fields)

To guarantee stability of the discrete system, we'll use a slope limiter.
Firedrake includes support for Kuzmin's vertex-based P1 limiter.

In [ ]:
limiter = firedrake.VertexBasedLimiter(Q)

A bit of helper code that will prevent warnings and give us a clue of what the solution is doing.

In [ ]:
parameters = {
    'form_compiler_parameters': {
        'quadrature_degree': 4
    }
}

def field_range(z):
    ζ = z.dat.data_ro[:]
    return ζ.min(), ζ.max()

We'll run the simulation in several phases, each of which will require a different timestep, so we'll wrap it up into one big function.

In [ ]:
from firedrake import max_value, min_value, sqrt
from plumes import numerics
def run_model(fields, final_time, dt, frac):
    num_steps = int(final_time / dt)
    zs = []
    integrator = numerics.Rosenbrock(equation, fields, **parameters)
    
    for step in range(num_steps):
        try:
            integrator.step(dt)
        except:
            return zs

        D, q, E, S = integrator.state.split()
        limiter.apply(D)
        limiter.apply(E)
        limiter.apply(S)
        S.interpolate(min_value(max_value(0, S), frac * D * s_a))
        E.interpolate(max_value(D * T_in, E))
        zs.append(integrator.state.copy(deepcopy=True))

        T = firedrake.interpolate(E / D, Q)
        s = firedrake.interpolate(S / D, Q)
        u = firedrake.interpolate(q / D, W)
        U = firedrake.interpolate(sqrt(inner(u, u)), Q)

        Dmin, Dmax = field_range(D)
        Tmin, Tmax = field_range(T)
        smin, smax = field_range(s)
        umin, umax = field_range(U)

        print(
            f'{dt * step / 60:6.1f}'
            f' | {Dmin:5.3f}, {Dmax:5.3f}'
            f' | {Tmin:5.3f}, {Tmax:5.3f}'
            f' | {smin:5.3f}, {smax:5.3f}'
            f' | {umin:5.3f}, {umax:5.3f}',
            sep=' ',
            flush=True
        )
        
    return zs

This function encapsulates some tedious busywork for running the simulation and saving the results to a file if it does not exist, and loading the results from that file if it does exist.

In [ ]:
from firedrake import DumbCheckpoint as Checkpoint
import os

def run_or_load_results(filename, fields, final_time, timestep, salt_frac):
    filestem = os.path.splitext(filename)[0]
    if os.path.exists(filename):
        zs = []
        z = firedrake.Function(Z)
        with Checkpoint(filestem, mode=firedrake.FILE_READ) as chk:
            steps, indices = chk.get_timesteps()
            for step in steps:
                chk.set_timestep(step)
                chk.load(z, name='z')
                zs.append(z.copy(deepcopy=True))
    else:
        zs = run_model(fields, final_time, timestep, salt_frac)
        with Checkpoint(filestem, mode=firedrake.FILE_CREATE) as chk:
            for step, z in enumerate(zs):
                chk.set_timestep(step)
                chk.store(z, name='z')
                
    return zs

Now we'll actually run the first phase.
This goes for about 7 hours of physical time before weird things start to happen that necessitate much smaller timesteps in the second phase.

In [ ]:
frac = Constant(0.25)
dt = 60.0
final_time = 7 * 60 * 60.0
zs = run_or_load_results('pine-island-phase1.h5', fields, final_time, dt, frac)

And some silly helper code to make animations.

In [ ]:
def make_animation(zs, Dmin, Dmax, umin, umax, fps):
    fig, axes = icepack.plot.subplots(ncols=2, sharex=True, sharey=True)
    for ax in axes:
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

    D, q, E, S = zs[-1].split()
    U = firedrake.project(sqrt(inner(q / D, q / D)), Q)
    colors_U = firedrake.tripcolor(U, num_sample_points=1, vmin=umin, vmax=umax, axes=axes[0])
    colors_D = firedrake.tripcolor(D, num_sample_points=1, vmin=Dmin, vmax=Dmax, axes=axes[1])
    
    axes[0].set_title('Speed')
    axes[1].set_title('Thickness')

    from matplotlib.animation import FuncAnimation
    def animate(z):
        D, q, E, S = z.split()
        U.project(sqrt(inner(q / D, q / D)))
        colors_U.set_array(U.dat.data_ro[:])
        colors_D.set_array(D.dat.data_ro[:])

    interval = 1e3 / fps
    return FuncAnimation(fig, animate, frames=zs, interval=interval)

In [ ]:
%%capture
animation = make_animation(zs, Dmin=0.0, Dmax=32.0, umin=0.0, umax=2.0, fps=30)

In [ ]:
from IPython.display import HTML
HTML(animation.to_html5_video())

For the second phase, we need to use a much smaller timestep.

In [ ]:
frac = Constant(0.25)
dt = 4.0
final_time = 60 * 60.0
zs = run_or_load_results('pine-island-phase2.h5', zs[-1], final_time, dt, frac)

In [ ]:
%%capture
animation = make_animation(zs, Dmin=0.0, Dmax=32.0, umin=0.0, umax=2.0, fps=30)

In [ ]:
HTML(animation.to_html5_video())

In [ ]:
frac = Constant(0.25)
dt = 2.0
final_time = 60 * 60.0
zs = run_or_load_results('pine-island-phase3.h5', zs[-1], final_time, dt, frac)

In [ ]:
%%capture
animation = make_animation(zs, Dmin=0.0, Dmax=32.0, umin=0.0, umax=2.0, fps=60)

In [ ]:
HTML(animation.to_html5_video())

In [ ]:
frac = Constant(0.25)
dt = 2.0
final_time = 30 * 60.0
zs = run_or_load_results('pine-island-phase4.h5', zs[-1], final_time, dt, frac)

In [ ]:
%%capture
animation = make_animation(zs, Dmin=0.0, Dmax=32.0, umin=0.0, umax=2.0, fps=30)

In [ ]:
HTML(animation.to_html5_video())